# Vision Language Model

SGLang supports vision language models in the same way as completion models. Here are some example models:

- [lmms-lab/llava-onevision-qwen2-7b-ov](https://huggingface.co/lmms-lab/llava-onevision-qwen2-7b-ov)
- [meta-llama/Llama-3.2-11B-Vision-Instruct](https://huggingface.co/meta-llama/Llama-3.2-11B-Vision-Instruct)


## Launch A Server

The following code is equivalent to running this in the shell:

```bash
python3 -m sglang.launch_server --model-path meta-llama/Llama-3.2-11B-Vision-Instruct \
 --port=30010 --chat-template=llama_3_vision
```

Remember to add `--chat-template=llama_3_vision` to specify the vision chat template, otherwise the server only supports text.

In [14]:
from sglang.utils import (
    execute_shell_command,
    wait_for_server,
    terminate_process,
    print_highlight,
)

embedding_process = execute_shell_command(
    """
    python3 -m sglang.launch_server --model-path meta-llama/Llama-3.2-11B-Vision-Instruct \
        --port=30010 --chat-template=llama_3_vision

"""
)

wait_for_server("http://localhost:30010")

## Use Curl

In [11]:
import subprocess, json

curl_command = """
curl http://localhost:30010/v1/chat/completions \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer None" \
  -d '{
    "model": "meta-llama/Llama-3.2-11B-Vision-Instruct",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "What’s in this image?"
          },
          {
            "type": "image_url",
            "image_url": {
              "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
            }
          }
        ]
      }
    ],
    "max_tokens": 300
  }'
"""

response = json.loads(subprocess.check_output(curl_command, shell=True))
print_highlight(response)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

[2024-10-31 20:41:39 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 6462, cache hit rate: 66.63%, token usage: 0.05, #running-req: 0, #queue-req: 0
[2024-10-31 20:41:40] INFO:     127.0.0.1:58650 - "POST /v1/chat/completions HTTP/1.1" 200 OK


100  1048  100   489  100   559    445    508  0:00:01  0:00:01 --:--:--   954


## ## Using OpenAI Compatible API

In [12]:
import base64
from openai import OpenAI

client = OpenAI(base_url="http://localhost:30010/v1", api_key="None")

def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

image_path = "boardwalk.jpeg"
base64_image = encode_image(image_path)

response = client.chat.completions.create(
    model="meta-llama/Llama-3.2-11B-Vision-Instruct",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What is in this image?",
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}"
                    },
                },
            ],
        }
    ],
    max_tokens=300
)

print_highlight(response.choices[0].message.content)


[2024-10-31 20:44:17 TP0] Prefill batch. #new-seq: 1, #new-token: 6463, #cached-token: 0, cache hit rate: 49.98%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-10-31 20:44:18 TP0] Decode batch. #running-req: 1, #token: 6471, token usage: 0.05, gen throughput (token/s): 0.21, #queue-req: 0
[2024-10-31 20:44:19 TP0] Decode batch. #running-req: 1, #token: 6511, token usage: 0.05, gen throughput (token/s): 33.76, #queue-req: 0
[2024-10-31 20:44:20 TP0] Decode batch. #running-req: 1, #token: 6551, token usage: 0.05, gen throughput (token/s): 33.67, #queue-req: 0
[2024-10-31 20:44:21] INFO:     127.0.0.1:49900 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(embedding_process)

## Chat Template

As mentioned before, if you do not specify a vision model's `chat-template`, the server uses Hugging Face's default template, which only supports text.

You can add your custom chat template by referring to the [custom chat template](../references/custom_chat_template.md).

We list popular vision models with their chat templates:

- [meta-llama/Llama-3.2-Vision](https://huggingface.co/meta-llama/Llama-3.2-11B-Vision-Instruct) uses `llama_3_vision`.
- [LLaVA-NeXT](https://huggingface.co/collections/lmms-lab/llava-next-6623288e2d61edba3ddbf5ff) uses `chatml-llava`.
- [llama3-llava-next](https://huggingface.co/lmms-lab/llama3-llava-next-8b) uses `llava_llama_3`.
- [llava-onevision](https://huggingface.co/lmms-lab/llava-onevision-qwen2-7b-ov) uses `chatml-llava`.
- [liuhaotian/llava-v1.5 / 1.6](https://huggingface.co/liuhaotian/llava-v1.5-13b) uses `vicuna_v1.1`.